In [24]:
# RAG implementation using LangChain with real world data


import os
import glob
import tiktoken
import numpy as np
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from pathlib import Path

from markitdown import MarkItDown

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from langchain_core.messages import SystemMessage, HumanMessage
import gradio as gr

In [25]:
# file input and output paths to convert and output in markdown format

input_dir = "knowledge-base-cihp"
output_dir="output"

In [26]:
def convert_tables_to_records(markdown_content):
    """
    Converts markdown tables to single-line key-value record format (optimized for RAG)
    """
    import re
    
    lines = markdown_content.split('\n')
    processed_lines = []
    
    i = 0
    while i < len(lines):
        line = lines[i]
        
        # Check if this line starts a table
        if (line.strip() and line.strip().startswith('|')) and \
           (i + 1 < len(lines) and re.match(r'^\s*\|\s*[-:]+\s*(\|\s*[-:]+\s*)*\|\s*$', lines[i + 1])):
            
            # Get the header row
            header_line = lines[i].strip().strip('|').strip()
            headers = [header.strip() for header in header_line.split('|')]
            
            # Find the end of the table
            j = i + 2  # Start after header and separator
            while j < len(lines) and lines[j].strip().startswith('|'):
                j += 1
            
            # Process each data row
            for k in range(i + 2, j):  # Skip header and separator
                data_line = lines[k].strip().strip('|').strip()
                data_cells = [cell.strip() for cell in data_line.split('|')]
                
                # Create a single record with key-value pairs
                record_parts = []
                for idx, (header, value) in enumerate(zip(headers, data_cells)):
                    if header and value:  # Only include if both exist
                        # Clean up the value to remove problematic characters
                        clean_value = value.replace('\n', ' ').replace('\r', ' ')
                        record_parts.append(f"{header.strip()}: {clean_value}")
                
                if record_parts:  # Only add if there's actual content
                    record_line = " | ".join(record_parts)
                    processed_lines.append(f"RECORD: {record_line}")
            
            i = j  # Skip the processed table lines
        else:
            processed_lines.append(line)
            i += 1
    
    return '\n'.join(processed_lines)

def convert_to_markdown(
    input_dir,
    output_dir=None,  # optional
    target_formats=(".docx", ".xlsx", ".pdf", ".pptx"),
):
    input_path = Path(input_dir)
    
    md = MarkItDown()

    for file_path in input_path.rglob("*"):
        if file_path.suffix in target_formats:
            try:
                result = md.convert(file_path)
                
                # Process the markdown content to convert tables to single-line records
                processed_markdown = convert_tables_to_records(result.markdown)
                
            except Exception as e:
                print(f"✗ Error converting {file_path.name}: {e}")
                continue

            output_file = file_path.parent / f"{file_path.stem}{file_path.suffix}.md"
            
            output_file.write_text(processed_markdown, encoding="utf-8")
            print(f"✓ Converted {file_path.name} → {output_file.name}")

In [27]:
convert_to_markdown(input_dir,output_dir)

✓ Converted Cool girls concept note for evaluation .docx → Cool girls concept note for evaluation .docx.md
✓ Converted Concept Note on Cool Girls Online Radio.docx → Concept Note on Cool Girls Online Radio.docx.md
✓ Converted employees.xlsx → employees.xlsx.md
✓ Converted NDR.pptx → NDR.pptx.md
✓ Converted CIHP Employee Policy Procedure Manual.pdf → CIHP Employee Policy Procedure Manual.pdf.md
✓ Converted SOP - Codes of Conduct.pdf → SOP - Codes of Conduct.pdf.md
✓ Converted SOP - HUMAN RESOURCES.pdf → SOP - HUMAN RESOURCES.pdf.md
✓ Converted SOP - IT Policy.pdf → SOP - IT Policy.pdf.md
✓ Converted SOP - Recruitment Policy Oct 2012.pdf → SOP - Recruitment Policy Oct 2012.pdf.md
✓ Converted SOP-PROCUREMENT GUIDELINES.pdf → SOP-PROCUREMENT GUIDELINES.pdf.md


In [28]:
MODEL = "gpt-4.1-nano"
DB_NAME = "cihp_vector_db"
load_dotenv(override=True)

True

In [29]:
# OpenAI API key

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [30]:
# How many characters in all the documents?

knowledge_base_path = input_dir + "/**/*.md"
files = glob.glob(knowledge_base_path, recursive=True)
print(f"Found {len(files)} files in the knowledge base")

entire_knowledge_base = ""

for file_path in files:
    with open(file_path, 'r', encoding='utf-8') as f:
        entire_knowledge_base += f.read()
        entire_knowledge_base += "\n\n"

print(f"Total characters in knowledge base: {len(entire_knowledge_base):,}")

Found 10 files in the knowledge base
Total characters in knowledge base: 435,932


In [31]:
# How many tokens in all the documents?

encoding = tiktoken.encoding_for_model(MODEL)
tokens = encoding.encode(entire_knowledge_base)
token_count = len(tokens)
print(f"Total tokens for {MODEL}: {token_count:,}")

Total tokens for gpt-4.1-nano: 107,708


In [32]:
# Load in everything in the knowledgebase using LangChain's loaders

folders = glob.glob(input_dir + "/*")

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print(f"Loaded {len(documents)} documents")

Loaded 10 documents


In [33]:
documents[0]

Document(metadata={'source': 'knowledge-base-cihp\\coolgirls\\Cool girls concept note for evaluation .docx.md', 'doc_type': 'coolgirls'}, page_content="Outline and guide for short concept note on evaluation of new program interventions proposed for implementation\n\n# **Introduction**\n\n·\xa0\xa0\xa0\xa0\xa0\xa0\xa0*Include the problem statement –*\xa0*what is the program issue or gap to be addressed by the intervention? Highlight who is affected, where, and why is it important to focus on the identified group or sub-population?*\n\n*Sub-Saharan Africa is grappling with a significant HIV/AIDS burden, particularly among adolescent girls and young women (AGYW). Despite comprising only 10% of the total population, AGYW account for about 25% of all HIV infections. UNAIDS reported that 6,200 young women aged 15-24 are infected with HIV on a weekly basis worldwide, highlighting the urgent need for targeted interventions. Various factors contribute to the heightened vulnerability of AGYW to 

In [34]:
# Divide into chunks using the RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"Divided into {len(chunks)} chunks")
print(f"First chunk:\n\n{chunks[1]}")

Divided into 582 chunks
First chunk:

page_content='*Sub-Saharan Africa is grappling with a significant HIV/AIDS burden, particularly among adolescent girls and young women (AGYW). Despite comprising only 10% of the total population, AGYW account for about 25% of all HIV infections. UNAIDS reported that 6,200 young women aged 15-24 are infected with HIV on a weekly basis worldwide, highlighting the urgent need for targeted interventions. Various factors contribute to the heightened vulnerability of AGYW to HIV, including socio-cultural barriers to comprehensive sexuality education, inter-generational and transactional sex practices, lack of schooling and economic empowerment, gender-based violence, and harmful traditional practices.*

* *What is the proposed solution? Has it been used before? If so, how (and where) was it used and how effective was it? (Useful to provide evidence for this - references). Why do you think the proposed intervention would be effective for the problem descr

In [35]:
# Pick an embedding model

#embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

if os.path.exists(DB_NAME):
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=DB_NAME)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 582 documents


In [36]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 582 vectors with 3,072 dimensions in the vector store


In [37]:
# Visualizing  - Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange', 'darkblue'][['coolgirls', 'ndr', 'sop', 'employees', 'coolgirls-radio'].index(t)] for t in doc_types]

In [38]:
# Visualizing in 2D
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [39]:
# Visualzing in 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

In [40]:
# Connect to Chroma; use Hugging Face all-MiniLM-L6-v2

# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)

In [41]:
# Seting up the 2 key LangChain objects: retriever and llm
# Also, the temperature

retriever = vectorstore.as_retriever()
llm = ChatOpenAI(temperature=0, model_name=MODEL)

In [42]:
SYSTEM_PROMPT_TEMPLATE = """
You are a knowledgeable, friendly assistant representing the company CIHP.
You are chatting with a user about CIHP.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Context:
{context}
"""

In [43]:
def answer_question(question: str, history):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context,history=history)
    response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    return response.content

In [44]:
# UI implementation with gradio

gr.ChatInterface(answer_question).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
